In [18]:
# Importer les librairies nécessaires
import cv2
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import sqlite3
from datetime import datetime
import matplotlib.pyplot as plt
import os

## 1. Préparation des données

In [25]:
plaque = 'train'
image_size = (224, 224)  # plus raisonnable

images = []

for filename in os.listdir(plaque):
    if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        path = os.path.join(plaque, filename)
        try:
            img = Image.open(path).convert('RGB')
            img = img.resize(image_size)
            img_array = np.array(img, dtype=np.float32) / 255.0  # Normalisé et plus léger
            images.append(img_array)
        except Exception as e:
            print(f"Erreur avec {filename} : {e}")

images = np.array(images)

print("Nombre d'images chargées :", len(images))
print("Taille de chaque image :", images[0].shape)


Nombre d'images chargées : 5622
Taille de chaque image : (224, 224, 3)


In [26]:

from PIL import Image

# Définir le chemin du répertoire contenant les images
plaque = "train"  # Remplacez par le chemin réel de votre dossier (ex. "MEMOIRE/test/images")
image_size = (224, 224)  # Taille raisonnable pour les images

images = []

# Vérifier si le répertoire existe
if not os.path.exists(plaque):
    raise FileNotFoundError(f"Le répertoire {plaque} n'existe pas. Vérifiez le chemin ou créez le dossier.")

# Charger toutes les images du répertoire
for filename in os.listdir(plaque):
    if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        path = os.path.join(plaque, filename)
        try:
            # Ouvrir l'image et la convertir en RGB
            img = Image.open(path).convert('RGB')
            # Redimensionner l'image
            img = img.resize(image_size, Image.Resampling.LANCZOS)  # Utilisation de LANCZOS pour une meilleure qualité
            # Convertir en tableau numpy et normaliser
            img_array = np.array(img, dtype=np.float32) / 255.0
            images.append(img_array)
        except Exception as e:
            print(f"Erreur avec {filename} : {e}")
            continue

# Convertir la liste en tableau numpy
if images:
    images = np.array(images)
    print("Nombre d'images chargées :", len(images))
    print("Taille de chaque image :", images[0].shape)
else:
    print("Aucune image n'a été chargée. Vérifiez le répertoire ou les fichiers.")

# Vérifier le répertoire courant pour débogage
print("Répertoire courant :", os.getcwd())

Nombre d'images chargées : 5622
Taille de chaque image : (224, 224, 3)
Répertoire courant : C:\Users\User\MEMOIRE


In [28]:
import os
import cv2
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

def load_and_preprocess_data(dataset_path, labels_dict=None):
    images = []
    image_paths = []
    labels = []

    # Convertir en chemin absolu
    absolute_path = os.path.abspath(dataset_path)
    
    # Vérifier si le répertoire existe
    if not os.path.exists(absolute_path):
        raise ValueError(f"Le répertoire {absolute_path} n'existe pas. Vérifiez le chemin ou créez le dossier.")

    # Lister tous les fichiers dans le répertoire
    for filename in os.listdir(absolute_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            file_path = os.path.join(absolute_path, filename)
            img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
            if img is None:
                print(f"Erreur : Impossible de charger l'image {file_path}. Vérifiez le fichier.")
                continue
            img = cv2.resize(img, (100, 30))  # Redimensionner
            img = img / 255.0  # Normalisation
            images.append(img)
            image_paths.append(file_path)
            # Attribuer un label
            if labels_dict and filename in labels_dict:
                label = labels_dict[filename]
                if label >= 36:  # Vérifier que le label ne dépasse pas 35
                    print(f"Attention : Label {label} pour {filename} dépasse 35. Réduit à 35.")
                    label = 35
                labels.append(label)
            else:
                labels.append(len(image_paths) - 1 % 36)  # Modulo 36 pour limiter à 35

    # Vérifier si des images ont été chargées
    if not images:
        raise ValueError(f"Aucune image n'a été chargée dans {absolute_path}. Vérifiez les fichiers ou les extensions.")

    # Convertir en tableau numpy et ajouter la dimension des canaux
    images = np.array(images).reshape(-1, 30, 100, 1)

    # Encoder les labels en one-hot (36 classes : 0-9, A-Z)
    labels = to_categorical(labels, num_classes=36)

    # Diviser les données
    return train_test_split(images, labels, test_size=0.2, random_state=42)

# Configuration du chemin du dataset
dataset_path = os.path.join("train")  # Chemin absolu corrigé
print("Répertoire courant :", os.getcwd())
print("Chemin du dataset spécifié :", dataset_path)

# Dictionnaire de labels personnalisés (optionnel)
labels_dict = None  # À définir si vous avez des labels spécifiques, sinon labels automatiques

try:
    X_train, X_test, y_train, y_test = load_and_preprocess_data(dataset_path, labels_dict)
    print(f"Données chargées avec succès. Forme de X_train : {X_train.shape}")
    print(f"Nombre total d'images chargées : {len(X_train) + len(X_test)}")
    print(f"Exemple de label encodé (premier échantillon) : {y_train[0]}")
except ValueError as e:
    print(e)

Répertoire courant : C:\Users\User\MEMOIRE
Chemin du dataset spécifié : train


IndexError: index 36 is out of bounds for axis 1 with size 36

## 2. Architecture CNN avec 10 couches

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(features, targets, test_size=0.05, random_state=42)

print("X_train.shape", X_train.shape)
print("X_valid.shape", X_valid.shape)
print("y_train.shape", y_train.shape)
print("y_valid.shape", y_valid.shape)

In [21]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(30, 100, 1)),  # Couche 1
    MaxPooling2D((2, 2)),  # Couche 2
    Conv2D(32, (3, 3), activation='relu'),  # Couche 3
    MaxPooling2D((2, 2)),  # Couche 4
    Conv2D(64, (3, 3), activation='relu'),  # Couche 5
    Conv2D(64, (3, 3), activation='relu'),  # Couche 6
    MaxPooling2D((2, 2)),  # Couche 7
    Flatten(),  # Couche 8
    Dense(128, activation='relu'),  # Couche 9
    Dropout(0.5),  # Couche 10
    Dense(36, activation='softmax')  # Sortie (36 classes pour 0-9, A-Z)
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

C:\Users\User\anaconda3\envs\machlearn\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 28, 98, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 14, 49, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 12, 47, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 6, 23, 32)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 4, 21, 64)      │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 2, 19, 64)      │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 1, 9, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 576)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 36)             │         4,644 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 143,492 (560.52 KB)

 Trainable params: 143,492 (560.52 KB)

 Non-trainable params: 0 (0.00 B)

## 3. Entraînement

In [22]:
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2, verbose=1)

NameError: name 'X_train' is not defined

## 4. Reconnaissance des plaques (sans YOLO)

In [ ]:
def recognize_plate(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.resize(gray, (100, 30))
    gray = gray / 255.0
    gray = np.expand_dims(gray, axis=(0, -1))
    prediction = model.predict(gray)
    char_indices = np.argmax(prediction, axis=1)
    chars = [chr(ord('0') + i) if i < 10 else chr(ord('A') + i - 10) for i in char_indices]
    plate_text = ''.join(chars)
    return plate_text

# Exemple avec une vidéo de test (remplacez par votre fichier vidéo)
cap = cv2.VideoCapture("test/test_video.mp4")  # Ajustez le chemin de la vidéo
if not cap.isOpened():
    print("Erreur : Impossible d'ouvrir la vidéo. Vérifiez le chemin.")
    exit()

fps = int(cap.get(cv2.CAP_PROP_FPS))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output_cnn.mp4', fourcc, fps, (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

start_time = datetime.now()
license_plates = set()
count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break
    count += 1
    print(f"Frame Number: {count}")
    plate_text = recognize_plate(frame)
    if plate_text:
        license_plates.add(plate_text)
        cv2.putText(frame, plate_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    out.write(frame)
    cv2.imshow("ANPR with CNN", frame)
    current_time = datetime.now()
    if (current_time - start_time).seconds >= 20:
        save_to_database(license_plates, start_time, current_time)
        start_time = current_time
        license_plates.clear()
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()

## 5. Sauvegarde dans la base de données

In [ ]:
def save_to_database(license_plates, start_time, end_time):
    conn = sqlite3.connect('licencePlates.db')
    cursor = conn.cursor()
    cursor.execute('CREATE TABLE IF NOT EXISTS LicencesPlates (id INTEGER PRIMARY KEY AUTOINCREMENT, start_time TEXT, end_time TEXT, licence_plate TEXT)')
    for plate in license_plates:
        cursor.execute('INSERT INTO LicencesPlates (start_time, end_time, licence_plate) VALUES (?, ?, ?)', (start_time.isoformat(), end_time.isoformat(), plate))
    conn.commit()
    conn.close()

## 6. Évaluation du modèle

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_accuracy:.4f}")
print(f"Test loss: {test_loss:.4f}")

# Visualisation
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

## Prérequis
- Installer les dépendances : `pip install tensorflow opencv-python numpy matplotlib sklearn`.
- Préparer un dossier 'test' avec des images annotées (ex. plate1.jpg, plate2.jpg, plate3.jpg).
- Avoir une vidéo 'test/test_video.mp4' pour le test en temps réel (optionnel, à créer ou remplacer).

## Améliorations possibles
- Ajouter une segmentation manuelle plus précise des caractères.
- Augmenter le nombre d'époques ou ajuster les hyperparamètres.
- Intégrer une validation croisée pour une robustesse accrue.